In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("match_samps.csv")
df2 = df.drop(["stats.name", "matchId", "assetsId", "matchType", "stats.deathType", "map", "stats.rank", "stats.playerId"], axis=1)
value_count = df2['gameMode'].value_counts()
df_value_counts = pd.DataFrame(value_count)
df_value_counts = df_value_counts.reset_index()
df_value_counts.columns = ['gameMode', 'num_games_played']
df3 = df2[df2['gameMode']=="solo"]
df3 = df3.drop(columns=["gameMode"])

def f(row):
    if row['stats.winPlace'] <= 5:
        val = 1
    else:
        val = 0

    return val
df3['top_five'] = df.apply(f, axis=1)
df3

,gameDuration,stats.DBNOs,stats.assists,stats.boosts,stats.damageDealt,stats.headshotKills,stats.heals,stats.killPlace,stats.killStreaks,stats.kills,...,stats.rideDistance,stats.roadKills,stats.swimDistance,stats.teamKills,stats.timeSurvived,stats.vehicleDestroys,stats.walkDistance,stats.weaponsAcquired,stats.winPlace,top_five
286,2214,0.0,0.0,0.0,100.000000,0.0,0.0,37.0,1.0,1.0,...,0.0000,0.0,0.0,0.0,1028.0,0.0,268.854000,0.0,63.0,0
287,2214,0.0,0.0,0.0,0.000000,0.0,0.0,90.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,789.0,0.0,28.123253,0.0,88.0,0
288,2214,0.0,0.0,4.0,427.098080,1.0,0.0,4.0,1.0,4.0,...,2323.3994,0.0,0.0,0.0,1229.0,0.0,1692.429000,4.0,21.0,0
289,2214,0.0,0.0,4.0,199.413650,0.0,0.0,28.0,1.0,1.0,...,0.0000,0.0,0.0,0.0,1647.0,0.0,1688.476400,0.0,31.0,0
290,2214,0.0,0.0,0.0,25.199999,0.0,0.0,44.0,0.0,0.0,...,4562.7593,0.0,0.0,0.0,1629.0,0.0,1043.243900,13.0,6.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20576,2150,0.0,0.0,0.0,43.537500,0.0,0.0,67.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,1294.0,0.0,964.296140,0.0,58.0,0
20577,2150,0.0,0.0,0.0,13.545000,0.0,0.0,89.0,0.0,0.0,...,0.0000,0.0,0.0,0.0,851.0,0.0,231.469160,0.0,89.0,0
20578,2150,0.0,0.0,3.0,438.063300,1.0,1.0,2.0,1.0,5.0,...,0.0000,0.0,0.0,0.0,1546.0,0.0,4143.685500,3.0,1.0,1
20579,2150,0.0,0.0,6.0,100.000000,0.0,0.0,24.0,1.0,1.0,...,0.0000,0.0,0.0,0.0,1838.0,0.0,4439.742000,0.0,22.0,0


In [3]:
df3.shape


(3829, 22)

In [4]:
target = df3["top_five"]
target_names = ["UNRanked", "Top5"]
data = df3.drop(["top_five", "stats.winPlace"], axis=1)
feature_names = data.columns
target

286      0
287      0
288      0
289      0
290      0
        ..
20576    0
20577    0
20578    1
20579    0
20580    0
Name: top_five, Length: 3829, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [6]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

SVC(kernel='linear')

In [7]:
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.957


In [8]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

              precision    recall  f1-score   support

    UNRanked       0.97      0.99      0.98       903
        Top5       0.68      0.49      0.57        55

    accuracy                           0.96       958
   macro avg       0.82      0.74      0.77       958
weighted avg       0.95      0.96      0.95       958



In [18]:
score_df = pd.DataFrame(columns=["Algorithm", "Test Accuracy", "Train Accuracy"])
score_df = score_df.append(pd.Series(["Support Vector Machines", model.score(X_test, y_test),  model.score(X_train, y_train)], index=["Algorithm", "Test Accuracy", "Train Accuracy"]), ignore_index=True)
score_df.to_csv('DataTrav/train_test_score.csv', mode='a', header=False)